<a href="https://colab.research.google.com/github/adw1029/CSCI_544/blob/main/HW1_CSCI544.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip setuptools wheel

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

ERROR: Operation cancelled by user


## Read Data

In [11]:
import os
current_directory = os.getcwd()

In [12]:
print("Current Directory:", current_directory)


Current Directory: /content


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
google_drive_directory = '/content/drive/My Drive/CSCI_544/hw1'
os.chdir(google_drive_directory)

In [15]:
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content/drive/My Drive/CSCI_544/hw1


In [16]:
gz_file_path = 'amz_data.gz'

## Keep Reviews and Ratings

In [17]:
file_path = 'amz_data.tsv'
data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])

<ipython-input-17-04b759942888>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])


In [18]:
print(data)

        star_rating                                        review_body
0                 5                                     Great product.
1                 5  What's to say about this commodity item except...
2                 5    Haven't used yet, but I am sure I will like it.
3                 1  Although this was labeled as &#34;new&#34; the...
4                 4                    Gorgeous colors and easy to use
...             ...                                                ...
2640347           4  I can't live anymore whithout my Palm III. But...
2640348           4  Although the Palm Pilot is thin and compact it...
2640349           4  This book had a lot of great content without b...
2640350           5  I am teaching a course in Excel and am using t...
2640351           5  A very comprehensive layout of exactly how Vis...

[2640352 rows x 2 columns]


Separate out review and rating, dropped null values

In [19]:
df_RevRat = data[data['star_rating'] != 3]

In [20]:
contains_null = df_RevRat[['star_rating', 'review_body']].isnull().any()
contains_null

star_rating    True
review_body    True
dtype: bool

In [21]:
df_RevRat = data.dropna(subset=['star_rating', 'review_body'])

In [22]:
df_RevRat

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [23]:
df_RevRat['star_rating'] = pd.to_numeric(df_RevRat['star_rating'], errors='coerce')


<ipython-input-23-835b95727621>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['star_rating'] = pd.to_numeric(df_RevRat['star_rating'], errors='coerce')


In [24]:
df_RevRat['star_rating'].dtypes

dtype('float64')

In [25]:
df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)

<ipython-input-25-940d36b9d541>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)


In [26]:
df_RevRat['Sentiment']

0          1
1          1
2          1
3          0
4          1
          ..
2640347    1
2640348    1
2640349    1
2640350    1
2640351    1
Name: Sentiment, Length: 2640255, dtype: int64

check if there's null values

In [27]:
contains_null = df_RevRat[['star_rating', 'review_body']].isnull().any()
contains_null

star_rating    False
review_body    False
dtype: bool

## 20000 samples from positive (100000) and negative (100000) reviews

In [28]:
pos_rev = df_RevRat[df_RevRat['Sentiment'] == 1].sample(100000, random_state=42)
neg_rev = df_RevRat[df_RevRat['Sentiment'] == 0].sample(100000, random_state=42)
df_small = pd.concat([pos_rev, neg_rev])

In [29]:
df_small.sample(3)

,star_rating,review_body,Sentiment
2475813,1.0,I purchased this ink to use in the Brother DCP...,0
1859239,3.0,Virtually all box cutters I have previously pu...,0
2311590,5.0,So far so good. I purchsed it 20 days ago. It ...,1


# Data Cleaning



# avg review length before cleaning

In [30]:
avglen_before = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_before)


Average text length: 315.554375


reviews to lower case

In [31]:
df_small['review_body'] = df_small['review_body'].str.lower()


remove URL and HTML

In [32]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')
html_pattern = re.compile('<.*?>')

In [33]:
def remove_html(text):
    return html_pattern.sub(r'', text)

# Function to remove URLs
def remove_urls(text):
    return url_pattern.sub(r'', text)

# Apply the functions to the review column
df_small['review_body'] = df_small['review_body'].apply(remove_html).apply(remove_urls)

remove non-alpha

In [34]:
df_small['review_body'] = df_small['review_body'].str.replace('[^a-zA-Z]', ' ', regex=True)

remove extra spaces

In [35]:
df_small['review_body'] = df_small['review_body'].str.replace(r'\s+', ' ', regex=True)

contractions

In [6]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 2.2 MB/s eta 0:00:00


In [36]:
import contractions


In [37]:
df_small['review_body'] = df_small['review_body'].apply(contractions.fix)

###average review length after cleaning

In [38]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 300.445865


# Pre-processing

In [39]:
from nltk.corpus import stopwords

### 3 sample reviews before pre-processing / Avg length

In [40]:
print(df_small['review_body'].head(3))

386895     i reviewed the packaging previously and gave s...
2514581    i got tired of searching my place for envelope...
785896               the best faber castell never dissapoint
Name: review_body, dtype: object


In [41]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 300.445865


### remove the stop words

In [42]:
nltk.download('stopwords')

stp_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [43]:
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stp_words]))


## perform lemmatization  

In [44]:
from nltk.stem import WordNetLemmatizer

In [45]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
lemmatizer = WordNetLemmatizer()
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


In [47]:
print(df_small['review_body'].head(3))

386895     reviewed packaging previously gave star replac...
2514581    got tired searching place envelope decided buy...
785896                   best faber castell never dissapoint
Name: review_body, dtype: object


In [48]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 184.625575


# TF-IDF Feature Extraction

#split testing (20%) and training (80%) set


In [49]:
vectorizer = TfidfVectorizer(max_features = 1000)
tfidf_mat = vectorizer.fit_transform(df_small['review_body'])

In [50]:
tfidf_mat

<200000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 3442741 stored elements in Compressed Sparse Row format>

# Perceptron

In [51]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [52]:
X = tfidf_mat
y = df_small['Sentiment']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
perceptron = Perceptron(random_state= 42).fit(X_train, y_train)

In [75]:
prec_train = perceptron.predict(X_train)
prec_test = perceptron.predict(X_test)

In [76]:
# Training Metrics
perc_train_acc = accuracy_score(y_train, prec_train)
perc_train_f1 = f1_score(y_train, prec_train, average = 'weighted')
perc_train_pre = precision_score(y_train, prec_train, average = 'weighted')
perc_train_recall = recall_score(y_train, prec_train, average = 'weighted')

# Testing Metrics
perc_test_acc = accuracy_score(y_test, prec_test)
perc_test_f1 = f1_score(y_test, prec_test, average = 'weighted')
perc_test_precision = precision_score(y_test, prec_test, average = 'weighted')
perc_test_recall = recall_score(y_test, prec_test, average = 'weighted')

In [77]:
print('Training Accuracy Score :', perc_train_acc)
print('Training F1 Score :', perc_train_f1)
print('Training Precision Score :',perc_train_pre)
print('Training Recall Score :', perc_train_recall)

print('\nTesting Accuracy Score :', perc_test_acc)
print('Testing F1 Score :', perc_test_f1)
print('Testing Precision Score :', perc_test_precision)
print('Testing Recall Score :', perc_test_recall)

Training Accuracy Score : 0.77421875
Training F1 Score : 0.7738967535868038
Training Precision Score : 0.7757949258899697
Training Recall Score : 0.77421875

Testing Accuracy Score : 0.7735
Testing F1 Score : 0.773123721439962
Testing Precision Score : 0.7753044083625289
Testing Recall Score : 0.7735


# SVM

In [58]:
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.linear_model import SGDClassifier



In [59]:
svm_sgd = SGDClassifier(loss='hinge', random_state=42)

In [60]:
svm_sgd.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [61]:
SVM_train = svm_sgd.predict(X_train)
SVM_test = svm_sgd.predict(X_test)

In [62]:
# Training Metrics
svm_train_acc = accuracy_score(y_train, SVM_train)
svm_train_f1 = f1_score(y_train, SVM_train, average = 'weighted')
svm_train_pre = precision_score(y_train, SVM_train, average = 'weighted')
svm_train_recall = recall_score(y_train, SVM_train, average = 'weighted')

# Testing Metrics
svm_test_acc = accuracy_score(y_test, SVM_test)
svm_test_f1 = f1_score(y_test, SVM_test, average = 'weighted')
svm_test_precision = precision_score(y_test, SVM_test, average = 'weighted')
svm_test_recall = recall_score(y_test, SVM_test, average = 'weighted')

In [63]:
print('Training Accuracy Score :', svm_train_acc)
print('Training F1 Score :', svm_train_f1)
print('Training Precision Score :',svm_train_pre)
print('Training Recall Score :', svm_train_recall)

print('\nTesting Accuracy Score :', svm_test_acc)
print('Testing F1 Score :', svm_test_f1)
print('Testing Precision Score :', svm_test_precision)
print('Testing Recall Score :', svm_test_recall)

Training Accuracy Score : 0.8335
Training F1 Score : 0.833443879004956
Training Precision Score : 0.8339527854376796
Training Recall Score : 0.8335

Testing Accuracy Score : 0.830775
Testing F1 Score : 0.8306830839695817
Testing Precision Score : 0.8314813429348136
Testing Recall Score : 0.830775


# Logistic Regression

In [64]:
from sklearn.linear_model import LogisticRegression


In [65]:
logReg = LogisticRegression(random_state=42)

In [66]:
logReg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [67]:
log_train = logReg.predict(X_train)
log_test = logReg.predict(X_test)

In [68]:
# Training Metrics
log_train_acc = accuracy_score(y_train, log_train)
log_train_f1 = f1_score(y_train, log_train, average = 'weighted')
log_train_pre = precision_score(y_train, log_train, average = 'weighted')
log_train_recall = recall_score(y_train, log_train, average = 'weighted')

# Testing Metrics
log_test_acc = accuracy_score(y_test, log_test)
log_test_f1 = f1_score(y_test, log_test, average = 'weighted')
log_test_precision = precision_score(y_test, log_test, average = 'weighted')
log_test_recall = recall_score(y_test, log_test, average = 'weighted')

In [69]:
print('Training Accuracy Score :', log_train_acc)
print('Training F1 Score :', log_train_f1)
print('Training Precision Score :',log_train_pre)
print('Training Recall Score :', log_train_recall)

print('\nTesting Accuracy Score :', log_test_acc)
print('Testing F1 Score :', log_test_f1)
print('Testing Precision Score :', log_test_precision)
print('Testing Recall Score :', log_test_recall)

Training Accuracy Score : 0.83701875
Training F1 Score : 0.8370067653265094
Training Precision Score : 0.8371191642107624
Training Recall Score : 0.83701875

Testing Accuracy Score : 0.833975
Testing F1 Score : 0.8339486426539803
Testing Precision Score : 0.8341799232089346
Testing Recall Score : 0.833975


# Naive Bayes

In [70]:
from sklearn.naive_bayes import MultinomialNB

In [71]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [72]:
nb_train = nb.predict(X_train)
nb_test = nb.predict(X_test)

In [73]:
# Training Metrics
nb_train_acc = accuracy_score(y_train, nb_train)
nb_train_f1 = f1_score(y_train, nb_train, average = 'weighted')
nb_train_pre = precision_score(y_train, nb_train, average = 'weighted')
nb_train_recall = recall_score(y_train, nb_train, average = 'weighted')

# Testing Metrics
nb_test_acc = accuracy_score(y_test, nb_test)
nb_test_f1 = f1_score(y_test, nb_test, average = 'weighted')
nb_test_precision = precision_score(y_test, nb_test, average = 'weighted')
nb_test_recall = recall_score(y_test, nb_test, average = 'weighted')

In [74]:
print('Training Accuracy Score :', nb_train_acc)
print('Training F1 Score :', nb_train_f1)
print('Training Precision Score :',nb_train_pre)
print('Training Recall Score :', nb_train_recall)

print('\nTesting Accuracy Score :', nb_test_acc)
print('Testing F1 Score :', nb_test_f1)
print('Testing Precision Score :', nb_test_precision)
print('Testing Recall Score :', nb_test_recall)

Training Accuracy Score : 0.808675
Training F1 Score : 0.808673612046302
Training Precision Score : 0.8086835822028811
Training Recall Score : 0.808675

Testing Accuracy Score : 0.807275
Testing F1 Score : 0.8072738490624599
Testing Precision Score : 0.8072810545914958
Testing Recall Score : 0.807275
